# Qwen3 Agent实战之解锁模型的智能交互能力

# 一、准备OpenAI API 服务
在这里，我使用ollama部署服务，提供一个openai API风格的服务，便于后面Qwen-Agent实践测试
## （一）安装 Ollama
访问 Ollama 官网，根据你的操作系统（Windows、macOS、Linux）下载安装包。
安装完成后，打开终端，输入 ollama --version 验证是否成功。

## （二）下载模型
运行以下命令拉取模型：
```bash
ollama pull qwen3:8b
```

模型会下载到本地（默认路径 ~/.ollama/models），大小约 16GB，需要等待十几分钟。

## （三）启动 Ollama 服务
输入以下命令启动 API 服务：
```bash
ollama serve
```

服务默认监听 http://localhost:11434，可以用浏览器访问这个地址测试（会返回状态信息）。

## 4.openai API服务测试

In [3]:
from openai import OpenAI

# 初始化客户端，指向 Ollama 的本地服务
client = OpenAI(
    base_url="http://localhost:11434/v1",  # Ollama API 地址
    api_key="qwen3-8b"  # Ollama 默认无需真实 API Key，填任意值即可
)

# 发送请求
response = client.chat.completions.create(
    model="qwen3:8B",  # 指定模型
    messages=[
        {"role": "system", "content": "你是一个智能AI助手。"},
        {"role": "user", "content": "你好，什么是大模型？"}
    ],
    temperature=0.7,  # 控制生成多样性
    max_tokens=512    # 最大生成 token 数
)

# 打印结果
print(response.choices[0].message.content)

<think>
嗯，用户问的是“什么是大模型？”，我需要先明确大模型的定义。大模型通常指的是大规模参数模型，比如像GPT-3、BERT这样的模型，它们有数百万到数十亿的参数。不过，用户可能不太清楚参数的具体含义，可能需要用更通俗的语言解释。

接下来，我得考虑用户可能的背景。他们可能对AI有一定的了解，但不确定大模型的具体概念，或者完全是个新手。所以，我需要用简单易懂的例子来说明，比如比较小模型和大模型的区别，比如用手机电池容量来比喻参数量。

然后，用户可能想知道大模型的应用场景。比如自然语言处理、图像识别、语音助手等，这些都是常见的应用领域。需要举一些实际的例子，比如翻译、写作文、生成图片，这样用户更容易理解。

另外，用户可能关心大模型的优势和挑战。优势方面，大模型在多个任务上表现优异，能够处理复杂任务，但缺点是需要大量算力和数据，训练成本高，还有可能产生偏见或错误信息。这部分需要客观说明，避免过于技术化。

还要注意用户可能的深层需求。他们也许想知道为什么大模型重要，或者如何使用这些模型。所以，在回答中可以提到实际应用案例，比如像通义千问这样的模型，帮助用户理解大模型的实际价值。

最后，确保回答结构清晰，分点说明，让用户容易跟随。同时，语言要口语化，避免专业术语过多，保持自然流畅。检查是否有遗漏的重要点，比如大模型的发展趋势或伦理问题，但可能用户暂时不需要这些，所以保持回答简洁但全面。
</think>

大模型（Large Model）是指拥有大量参数（通常上亿甚至数十亿）的深度学习模型。这类模型通过海量数据训练，能够捕捉复杂的模式和关系，从而在多种任务中表现出强大的能力。以下是关于大模型的几个关键点：

---

### **1. 什么是“参数”？**
参数是模型内部用于学习数据特征的变量。参数越多，模型的表达能力越强，可以处理更复杂的任务。例如：
- **小模型**：参数量可能只有几百万，适合简单任务（如分类）。
- **大模型**：参数量可达数十亿，能处理更复杂的任务（如理解自然语言、生成文本等）。

---

### **2. 大模型的核心特点**
- **强大的泛化能力**：能将


# 二、安装Qwen-Agent

```bash
pip install -U "qwen-agent[gui,rag,code_interpreter,mcp]"
```

# 三、Qwen-Agent 使用示例

In [6]:
from qwen_agent.agents import Assistant

# 定义 LLM
llm_cfg = {
    'model': 'qwen3:8b',
    # 使用自定义 OpenAI API 兼容的端点
    'model_server': 'http://localhost:11434/v1',  # api_base
    'api_key': 'EMPTY',
}

# 定义工具
tools = [
    {'mcpServers': {
        'time': {
            'command': 'uvx',
            'args': ['mcp-server-time', '--local-timezone=Asia/Shanghai']
        },
        "fetch": {
            "command": "uvx",
            "args": ["mcp-server-fetch"]
        }
    }},
    'code_interpreter',  # 内置工具
]

# 定义代理
bot = Assistant(llm=llm_cfg, function_list=tools)

# 流式生成
messages = [{'role': 'user', 'content': 'https://qwenlm.github.io/blog/ 介绍 Qwen 的最新发展'}]
for responses in bot.run(messages=messages):
    pass
print(responses)

2025-06-02 21:36:15,215 - mcp_manager.py - 141 - INFO - Initializing MCP tools from mcp servers: ['time', 'fetch']
2025-06-02 21:36:15,219 - mcp_manager.py - 370 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: time
2025-06-02 21:36:15,844 - mcp_manager.py - 370 - INFO - Initializing a MCP stdio_client, if this takes forever, please check the config of this mcp server: fetch


[{'role': 'assistant', 'content': '<think>\nOkay, the user wants information about the latest developments in Qwen from the provided URL. First, I need to fetch the content from the website. The URL is https://qwenlm.github.io/blog/. Since the user mentioned "介绍 Qwen 的最新发展", which translates to introducing the latest developments of Qwen, I should retrieve the blog content.\n\nI\'ll use the fetch function to get the webpage. The parameters required are the URL, and maybe set max_length to 5000 as default. Since the user might want a summary, I\'ll fetch the content and then extract the relevant information. However, the fetch function can return either raw HTML or markdown. Since the user might prefer a clean summary, I\'ll set raw to false to get markdown content, which is easier to parse.\n\nWait, but the fetch function\'s raw parameter defaults to false, so maybe I don\'t need to specify it. But to be safe, maybe include it. Also, check if the URL is correct. The user provided the U

## 第二轮测试

In [7]:
# 流式生成
messages = [{'role': 'user', 'content': '明天的日期是？'}]
for responses in bot.run(messages=messages):
    pass
print(responses)

[{'role': 'assistant', 'content': '<think>\n好的，用户问的是“明天的日期是？”。我需要先确定用户所在的时区，因为不同的时区日期可能不同。根据提供的工具，可以使用time-get_current_time函数，但需要指定时区。然而，用户没有提供时区信息，所以按照函数描述，默认会使用Asia/Shanghai作为本地时区。\n\n首先，我需要获取当前的时间，然后计算明天的日期。不过，可能更直接的是直接获取明天的日期。但根据函数功能，time-get_current_time可能只返回当前时间，所以可能需要先获取当前时间，然后加一天。或者，可能需要另一个方法，但工具中没有日期加减的函数。因此，可能需要调用time-get_current_time函数，然后在结果上加一天。\n\n不过，用户的问题是关于日期的，而time-get_current_time返回的是当前时间，包括日期和时间。所以，如果当前时间是2023-10-05，那么明天的日期就是2023-10-06。因此，我需要先调用time-get_current_time，使用Asia/Shanghai时区，然后将日期加一天，再返回结果。\n\n但用户可能只需要日期部分，不需要时间。因此，步骤应该是：获取当前时间，然后计算明天的日期。不过，可能需要考虑时区的问题，确保正确性。例如，如果用户所在的时区是UTC+8，那么当前时间是下午某个时间，而明天的日期可能已经过了。不过，通常来说，获取当前时间后，加一天的日期部分即可。\n\n所以，我需要调用time-get_current_time函数，参数时区为Asia/Shanghai，然后处理返回的结果，得到明天的日期。或者，可能直接调用该函数，并返回当前日期的下一天。但具体实现可能需要代码来处理日期计算。不过，工具中的函数可能直接返回当前时间，因此需要手动计算。\n\n或者，可能用户的问题只需要当前日期的下一天，而不需要具体时间。因此，调用time-get_current_time函数，得到当前日期，然后加一天。例如，如果当前日期是2023-10-05，那么明天就是2023-10-06。这时候可能需要使用code_interpreter来执行日期计算。例如，用Python的datetime模块来处理。\n\n不过，根据工具中的函数，

## 第三轮测试

In [9]:
# 流式生成
messages = [{'role': 'user', 'content': '计算2的10次方'}]
for responses in bot.run(messages=messages):
    pass
print(responses)

[{'role': 'assistant', 'content': '<think>\n好的，用户让我计算2的10次方。这个问题看起来不难，但我要仔细确认用户的需求。首先，2的10次方就是2乘以自己10次，结果应该是1024。不过，用户可能希望看到详细的步骤，或者他们可能对指数运算不太确定。也有可能用户是在测试我的计算能力，或者他们需要这个结果用于某个具体的应用场景，比如编程、数学问题或者日常计算。\n\n我需要考虑是否需要用代码来执行这个计算，以确保准确性。虽然手动计算很容易，但使用代码解释器可以验证结果，并且展示过程。比如，用Python的话，可以直接写2**10，或者用循环相乘。不过，对于这样的简单问题，直接给出结果可能更高效。但用户可能希望看到更详细的解释，或者他们可能想了解不同的计算方法。\n\n另外，用户可能没有明确说明是否需要步骤，所以最好提供结果的同时，简要说明过程。比如，2^10等于1024，这是常见的计算机科学中的数值，比如1024字节等于1千字节。这样用户不仅得到答案，还能了解其应用场景。不过，如果用户只需要结果，直接回答即可。但为了全面，我应该提供结果和简要的解释，确保满足不同需求。\n</think>\n\n2的10次方等于1024。\n\n计算过程：\n2 × 2 × 2 × 2 × 2 × 2 × 2 × 2 × 2 × 2\n= 1024\n\n这个结果在计算机科学中很有意义，例如：\n- 1024字节 = 1千字节（KB）\n- 1024兆字节（MB）= 1千兆字节（GB）'}]
